### All of Statistics - Chapter 8 Exercise 3
Let X1, . . . ,Xn ∼ t3, n = 25
Let θ = T(F) = (q.75 − q.25)/1.34 where qp denotes the p_{th} quantile
### Requirements:
- Do a simulation to compare the coverage and length of the following confidence intervals for θ:
    1. Normal interval with standard error from the bootstrap
    2. bootstrap percentile interval, and
    3. pivotal bootstrap interval.

In [3]:
import numpy as np
from scipy.stats import norm, t
import plotly.express as px
import plotly.graph_objects as go

### Task: Initialization

In [4]:
T_of_F = lambda sample_: (np.quantile(sample_, .75) - np.quantile(sample_, .25)) / 1.34

### Task: Let $$X1, . . . ,Xn ∼ t3, n = 25$$

In [5]:
n = 25
sample = np.sort(t.rvs(3, size = n))

### Task:
Construct the three types of bootstrap 95% intervals for $$T(F)$$ from the data $$X_1, . . . ,X_n$$.

In [8]:
bootstrap_repetitions = 9999
bootstrap_estimations = list()
for i in range(bootstrap_repetitions):
    bootstrap_sample = np.random.choice(sample, size = len(sample), replace = True)
    bootstrap_estimations.append(T_of_F(bootstrap_sample))
bootstrap_estimations = np.sort(bootstrap_estimations)
theta_hat = T_of_F(sample)
se_hat = np.array(bootstrap_estimations).std()
print(f'Plug-in estimate for theta: theta_hat = {theta_hat}')
print(f'Bootstrap std. error estimate = {se_hat}')

alpha = 0.05
# Normal method
z = norm.ppf(1-alpha/2)
normal_upper_bound = theta_hat + se_hat * z
normal_lower_bound = theta_hat - se_hat * z
print(f'Normal method CI:({normal_lower_bound}, {normal_upper_bound})')
# Percentile method
percentile_upper_bound = np.quantile(bootstrap_estimations, 1 - alpha/2)
percentile_lower_bound = np.quantile(bootstrap_estimations, alpha/2)
print(f'Percentile method CI: ({percentile_lower_bound}, {percentile_upper_bound})')
# Pivotal method
pivotal_lower_bound = 2*theta_hat - np.quantile(bootstrap_estimations, 1-alpha/2)
pivotal_upper_bound = 2*theta_hat - np.quantile(bootstrap_estimations, alpha/2)
print(f'Pivotal method CI: ({pivotal_lower_bound}, {pivotal_upper_bound})')

Plug-in estimate for theta: theta_hat = 1.1572049013584185
Bootstrap std. error estimate = 0.2983673990829605
Normal method CI:(0.5724155449949268, 1.74199425772191)
Percentile method CI: (0.3850859617718756, 1.5353152182075787)
Pivotal method CI: (0.7790945845092583, 1.9293238409449613)


### Task: Repeat this whole thing many times and estimate the true coverage of the three intervals.

In [7]:
num_repeats = 9999
coverage = dict(
    normal = 0,
    percentile = 0,
    pivotal = 0
)

for _ in range(num_repeats):
    bootstrap_sample = np.random.choice(sample, size = len(sample), replace = True)
    bootstrap_sample_theta_hat = T_of_F(bootstrap_sample)

    if normal_lower_bound < bootstrap_sample_theta_hat < normal_upper_bound:
        coverage['normal'] += 1
    if pivotal_lower_bound < bootstrap_sample_theta_hat < pivotal_upper_bound:
        coverage['pivotal'] += 1
    if percentile_lower_bound < bootstrap_sample_theta_hat < percentile_upper_bound:
        coverage['percentile'] += 1

coverage['normal'] /= num_repeats
coverage['percentile'] /= num_repeats
coverage['pivotal'] /= num_repeats

for method, coverage_pct in coverage.items():
    print(f'The {method} method has a coverage of {np.round(coverage_pct,4)}%')

The normal method has a coverage of 0.9291%
The percentile method has a coverage of 0.9421%
The pivotal method has a coverage of 0.779%
